# Online Tensor Decomposition
#### to optimize online tensor decomposition (streaming analysis)

In [1]:
import time
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition.candecomp_parafac import initialize_factors, unfolding_dot_khatri_rao, KruskalTensor

In [266]:
# for sample video
from cv2 import VideoWriter, VideoWriter_fourcc

def make_video(tensor, filename):
    start = time.time()
    height = tensor.shape[1]
    width = tensor.shape[2]
    FPS = 24

    fourcc = VideoWriter_fourcc(*'MP42')
    video = VideoWriter(filename, fourcc, float(FPS), (width, height))

    for frame in tensor:
        video.write(np.uint8(frame))
    video.release()
    print('created', filename, time.time()-start)

In [267]:
def construct_tensor(factors):
    weights = tl.ones(factors[0].shape[1])
    est_tensor = tl.kruskal_to_tensor((weights, factors))
    return est_tensor
    
def print_tensor(X, n_digit=1):
    print(np.round(X, n_digit))
    
def compare_tensors(A, B):
    print('||A-B||:', tl.norm(A - B))
    
def create_tensor_stream(X, start_to_stream, batch_sizes):
    total_batch_size = np.sum(batch_sizes)
    if X.shape[0] != start_to_stream + total_batch_size:
        raise ValueError('Total batch size should be the size of streaming part of the tensor.')
    
    X_stream = [X[:start_to_stream]]
    batch_start = start_to_stream
    for batch_size in batch_sizes:
        batch_end = batch_start + batch_size
        X_stream.append(X[batch_start:batch_end])
        batch_start = batch_end
    return np.asarray(X_stream)
    
def get_KhatriRao(factors):
    n_dim = len(factors)
    lefts = [factors[n_dim-1]]
    rights = [factors[0]]
    if n_dim > 2:
        for mode in range(1, n_dim-1):
            lefts.append(tl.tenalg.khatri_rao((lefts[mode-1], factors[n_dim-mode-1])))
            rights.append(tl.tenalg.khatri_rao((factors[mode], rights[mode-1])))
            
    K = lefts.copy()
    K[0] = lefts[n_dim-2]
    K.append(rights[n_dim-2].copy())
    if n_dim > 2:
        for mode in range(1, n_dim-1):
            K[mode] = tl.tenalg.khatri_rao((lefts[n_dim-mode-2], rights[mode-1]))
    return K

def get_KhatriRao_except0(factors):
    n_dim = len(factors)
    lefts = np.empty((n_dim), dtype=object)
    rights = np.empty((n_dim), dtype=object)
    K = np.empty((n_dim), dtype=object)
    
    lefts[1] = factors[n_dim-1]
    rights[1] = factors[1]
    if n_dim > 3:
        for mode in range(2, n_dim-1):
            lefts[mode] = tl.tenalg.khatri_rao((factors[n_dim-mode], lefts[mode-1]))
            rights[mode] = tl.tenalg.khatri_rao((rights[mode-1], factors[mode]))
            
    K[1] = lefts[n_dim-2]
    K[n_dim-1] = rights[n_dim-2]
    if n_dim > 3: 
        for mode in range(2, n_dim-1):
            K[mode] = tl.tenalg.khatri_rao((rights[mode-1], lefts[n_dim-mode-1]))
    return K
    
def get_Hadamard(factors):
    rank = factors[0].shape[1]
    H = tl.tensor(np.ones((rank, rank)))
    for factor in factors:
        H = H * tl.dot(tl.transpose(factor), factor)
    return H

## Online CP

In [302]:
def online_cp(factors_old, X_old, X_new, rank, P, Q, n_iter=1, mu=1, verbose=False, transformed=False):
    weights = tl.ones(rank)
    if verbose:
        X = tl.tensor(np.concatenate((X_old, X_new)))
    n_dim = tl.ndim(X_old)
    U = factors_old.copy()
    
    if not transformed:
        K = get_KhatriRao_except0(factors_old)
    H = get_Hadamard(factors_old[1:])
        
    for i in range(n_iter):
        # temporal mode for A1
        if not transformed:
            mttkrp = tl.dot(tl.unfold(X_new, 0), tl.tenalg.khatri_rao((U[1], K[1])))
        else:
            # for higher accracy, lower speed
            mttkrp_parts = []
            for r in range(rank):
                component = tl.tenalg.multi_mode_dot(X_new, [f[:, r] for f in U], skip=0)
                mttkrp_parts.append(component)
            mttkrp = np.stack(mttkrp_parts, axis=1)
        
        A1 = tl.transpose(tl.solve(tl.transpose(H), tl.transpose(mttkrp)))

        # non-temporal mode
        for mode in range(1, n_dim):
            
            if not transformed:
                dP = tl.dot(tl.unfold(X_new, mode), tl.tenalg.khatri_rao((A1, K[mode])))
                UTU  = tl.dot(tl.transpose(U[mode]), U[mode])
                dQ = tl.dot(tl.transpose(A1), A1) * H / UTU
                
                U[mode] = tl.transpose(tl.solve(tl.transpose(mu*Q[mode] + dQ), tl.transpose(mu*P[mode] + dP)))
#                 K = updated K due to non-temporal mode change
#                 H = H_mode * tl.dot(tl.transpose(U[mode]), U[mode]) / UTU
                P[mode] = P[mode] + dP
                Q[mode] = Q[mode] + dQ
            else:
                U1 = U.copy()
                U1[0] = A1
                
                H_mode  = H / tl.dot(tl.transpose(U[mode]), U[mode])
                V = (mu * tl.dot(tl.transpose(U[0]), U[0]) + tl.dot(tl.transpose(A1), A1)) * H_mode
                
                mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, U), mode)
                mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
                
                U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mu*mttkrp0 + mttkrp1)))
                H = H_mode * tl.dot(tl.transpose(U[mode]), U[mode])
                
        # temporal mode for A0
        if transformed:
            mttkrp = unfolding_dot_khatri_rao(X_old, (None, U), 0)
#             mttkrp = tl.dot(tl.unfold(X_old, 0), tl.tenalg.khatri_rao((U[1], K[1])))
            U[0] = tl.transpose(tl.solve(tl.transpose(H), tl.transpose(mttkrp)))
            
        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return (KruskalTensor((weights, U)), P, Q)

## DTD

In [303]:
def dtd(factors_old, X_old, X_new, rank, n_iter=1, mu=1, verbose=False):

    weights = tl.ones(rank)
    if verbose:
        X = tl.tensor(np.concatenate((X_old, X_new)))
    n_dim = tl.ndim(X_old)
    U = factors_old.copy()
    
    for i in range(n_iter):
        # temporal mode for A1
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, U), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        # non-temporal mode
        for mode in range(1, n_dim):
            U1 = U.copy()
            U1[0] = A1
            V = tl.tensor(np.ones((rank, rank)))
            W = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                factor_old = factors_old[j]
                if j != mode:
                    W = W * tl.dot(tl.transpose(factor_old), factor)
                    if j == 0:
                        V = V * (mu*tl.dot(tl.transpose(factor), factor) + tl.dot(tl.transpose(A1), A1))
                    else:
                        V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp0 = mu * tl.dot(factors_old[mode], W)
            mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
            U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        # temporal mode for A0
        V = tl.tensor(np.ones((rank, rank)))
        W = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            factor_old = factors_old[j]
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
                W = W * tl.dot(tl.transpose(factor_old), factor)
        mttkrp = tl.dot(factors_old[0], W)
        U[0] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return KruskalTensor((weights, U))

## Online Tensor Decomposition
* `onlinecp`, `transformed_onlinecp`, `dtd`

In [312]:
def online_tensor_decomposition(X_stream, X, rank, n_iter=1, mu=1, verbose=False, method='onlinecp'):
    if method == 'onlinecp':
        onlinecp = True
        transformed = False
    elif method == 'transformed_onlinecp':
        onlinecp = True
        transformed = True
    elif method == 'dtd':
        onlinecp = False
    else:
        raise ValueError('The method does not exist.')        
        
    X_old = X_stream[0]
    n_dim = tl.ndim(X_old)

    start = time.time()
    (weights, factors) = parafac(X_old, rank, init='random')
    print('making init decompositnlinecp, transformed_onlinecp, dtdion result:', time.time()-start)
    
    if verbose:
        X_est = construct_tensor(factors)
        compare_tensors(X_old, X_est)
    
    if onlinecp:
        start = time.time()
        print('\n >> onlinecp rank-{} n_iter-{} mu-{} transformed-{}'.format(rank, n_iter, mu, transformed))
        K = get_KhatriRao_except0(factors)
        H = get_Hadamard(factors)

        P = np.empty((n_dim), dtype=object)
        Q = np.empty((n_dim), dtype=object)
    
        for mode in range(1, n_dim):
            P[mode] = tl.dot(tl.unfold(X_old, mode), tl.tenalg.khatri_rao((factors[0], K[mode])))
            Q[mode] = H / tl.dot(tl.transpose(factors[mode]), factors[mode])
        print('init_time:', time.time()-start)
    else:
        print('\n >> dtd rank-{} n_iter-{} mu-{}'.format(rank, n_iter, mu))
        
        
    for i, X_new in enumerate(X_stream[1:]):
        
        start = time.time()
        if onlinecp:
            ((weights, factors), P, Q) = online_cp(factors, X_old, X_new, rank, P, Q, n_iter=n_iter, mu=mu, verbose=False, transformed=transformed)
        else:
            (weights, factors) = dtd(factors, X_old, X_new, rank, n_iter=n_iter, mu=mu, verbose=False)
        
        U = factors.copy()
        U[0] = U[0][-X_new.shape[0]-1:-1]
        dX_est = construct_tensor(U)
        
        print('{}th_iter:'.format(i+1), time.time()-start, tl.norm(X_new-dX_est))
        
        X_old = np.concatenate((X_old, X_new))
        
        if verbose:
            X_est = construct_tensor(factors)
            compare_tensors(X_old, X_est)
    
    weights = tl.ones(rank)
    return KruskalTensor((weights, factors))

### Single tensor example

In [284]:
tensor = tl.tensor(np.arange(12000000, dtype='d').reshape((500, 40, 30, 20)))
tensor_old = tensor[:300,:,:,:]
tensor_new = tensor[300:,:,:,:]
rank = 4
n_dim = tl.ndim(tensor)

start = time.time()
(weights, factors_old) = parafac(tensor_old, rank)
print('making prev decomposition result:', time.time()-start)

making prev decomposition result: 2.4052579402923584


In [305]:
start = time.time()
K = get_KhatriRao_except0(factors_old)
H = get_Hadamard(factors_old)

P = np.empty((n_dim), dtype=object)
Q = np.empty((n_dim), dtype=object)
for mode in range(1, n_dim):
    P[mode] = tl.dot(tl.unfold(tensor_old, mode), tl.tenalg.khatri_rao((factors_old[0], K[mode])))
    Q[mode] = H / tl.dot(tl.transpose(factors_old[mode]), factors_old[mode])
print('init time:', time.time()-start)

start = time.time()
print('\n >> online_cp start')
((weights, factors), P, Q) = online_cp(factors_old, tensor_old, tensor_new, rank, P, Q, n_iter=10, mu=0.95, verbose=False, transformed=True)

print('exec time:', time.time()-start)
tensor_est = construct_tensor(factors)
compare_tensors(tensor, tensor_est)
print_tensor(np.asarray((tensor, tensor_est))[:,0,0,0,:10])


start = time.time()
print('\n >> online_cp start')
((weights, factors), P, Q) = online_cp(factors_old, tensor_old, tensor_new, rank, P, Q, mu=1, verbose=False, transformed=False)

print('exec time:', time.time()-start)
tensor_est = construct_tensor(factors)
compare_tensors(tensor, tensor_est)
print_tensor(np.asarray((tensor, tensor_est))[:,0,0,0,:10])


start = time.time()
print('\n >> dtd start')
(weights, factors) = dtd(factors_old, tensor_old, tensor_new, rank, mu=0.7, verbose=False)

print('exec time:', time.time()-start)
tensor_est = construct_tensor(factors)
compare_tensors(tensor, tensor_est)
print_tensor(np.asarray((tensor, tensor_est))[:,0,0,0,:10])

init time: 0.20088553428649902

 >> online_cp start
exec time: 8.059545040130615
||A-B||: 817.8731788480396
[[ 0.   1.   2.   3.   4.   5.   6.   7.   8.   9. ]
 [ 1.1  2.   3.   4.   5.   6.   7.   8.   9.  10. ]]

 >> online_cp start
exec time: 0.34842658042907715
||A-B||: 1248.571747776511
[[ 0.   1.   2.   3.   4.   5.   6.   7.   8.   9. ]
 [ 2.1  3.1  4.1  5.   6.   7.   8.   9.  10.  11. ]]

 >> dtd start
exec time: 0.42242932319641113
||A-B||: 821.7190841011488
[[ 0.   1.   2.   3.   4.   5.   6.   7.   8.   9. ]
 [ 1.1  2.1  3.1  4.1  5.1  6.1  7.1  8.1  9.1 10.1]]


### Load Sample Video Dataset

In [214]:
import csv
X = tl.tensor(np.zeros([205, 240, 320, 3], dtype='d'))

for i in range(41):
    start = time.time()
    with open('../Data/sample_video/data/video{}.tensor'.format(i)) as file:
        reader = csv.reader(file, delimiter='\t')    
        for row in reader:
            indices = [[index] for index in np.int64(np.asarray(row[:-1]))-1]
            X[tuple(indices)] = np.double(row[-1])
    print('>> sample_video{} loaded '.format(i), time.time() - start)

>> sample_video0 loaded  34.83012628555298
>> sample_video1 loaded  39.70965218544006
>> sample_video2 loaded  35.30497646331787
>> sample_video3 loaded  36.05091404914856
>> sample_video4 loaded  34.220033407211304
>> sample_video5 loaded  33.99099278450012
>> sample_video6 loaded  35.67768120765686
>> sample_video7 loaded  35.36756372451782
>> sample_video8 loaded  35.56985592842102
>> sample_video9 loaded  36.910553216934204
>> sample_video10 loaded  38.572246074676514
>> sample_video11 loaded  37.32305383682251
>> sample_video12 loaded  36.940138816833496
>> sample_video13 loaded  37.76074719429016
>> sample_video14 loaded  34.49615812301636
>> sample_video15 loaded  35.71461892127991
>> sample_video16 loaded  33.55713391304016
>> sample_video17 loaded  34.04530668258667
>> sample_video18 loaded  35.99349069595337
>> sample_video19 loaded  37.97364521026611
>> sample_video20 loaded  35.04505109786987
>> sample_video21 loaded  35.292919397354126
>> sample_video22 loaded  34.79817509

In [215]:
make_video(X, './sample_video/original.avi')

created ./sample_video/original.avi 0.40268707275390625


### Usage for Online Tensor Decomposition
* Create a tensor stream (sum of batch sizes should match with total size of the tensor)
  * `create_tensor_stream(tensor, start_to_stream, batch_sizes)`
* Invoke online tensor decomposition
  * `online_tensor_decomposition(tensor stream, original tensor, rank, verbose, method)`
* Construct an estimated tensor w. factors, leaving weights (=identity matrix)
  * `construct_tensor(factors)`

In [297]:
X_stream = create_tensor_stream(X, start_to_stream=10, batch_sizes=np.full((39), 5, dtype=int))

In [316]:
# %%capture cap --no-stderr
rank = 50
(weights, factors) = online_tensor_decomposition(X_stream, X, rank, verbose=False, method='onlinecp')

X_est = construct_tensor(factors)
compare_tensors(X, X_est)
print_tensor(np.asarray((X, X_est))[:,100,100,100:110,0])
make_video(X_est, './sample_video/online_cp-{}.avi'.format(rank))

# with open('./sample_video/online_cp.txt', 'w') as f:
#     f.write(cap.stdout)

making init decomposition result: 106.40103101730347

 >> onlinecp rank-50 n_iter-1 mu-1 transformed-False
init_time: 0.8317220211029053
1th_iter: 1.330268383026123 13458.49382038362
2th_iter: 1.170792579650879 16049.495744978942
3th_iter: 1.0813071727752686 19860.80454647391
4th_iter: 1.2328388690948486 20169.703658974646
5th_iter: 1.1033875942230225 18334.04898101678
6th_iter: 1.250990390777588 17648.529632458143
7th_iter: 1.0947942733764648 17250.99671574091
8th_iter: 1.1017801761627197 16695.466187529808
9th_iter: 1.026233434677124 15856.85174719606
10th_iter: 1.0523748397827148 15287.770694490788
11th_iter: 1.1287438869476318 17994.462725348476
12th_iter: 0.9907996654510498 17466.368504698326
13th_iter: 1.3017864227294922 16327.552330004923
14th_iter: 1.1327571868896484 15681.349965548558
15th_iter: 1.0973279476165771 15937.560907904111
16th_iter: 1.114393711090088 15004.870673274756
17th_iter: 1.234917163848877 14691.556178762989
18th_iter: 1.3388853073120117 19147.931326036127
1

In [317]:
# %%capture cap --no-stderr
rank = 50
(weights, factors) = online_tensor_decomposition(X_stream, X, rank=50, n_iter=1, verbose=False, method='transformed_onlinecp')

X_est = construct_tensor(factors)
compare_tensors(X, X_est)
print_tensor(np.asarray((X, X_est))[:,100,100,100:110,0])
make_video(X_est, './sample_video/transformed_online_cp-{}.avi'.format(rank))

# with open('./sample_video/transformed_online_cp.txt', 'w') as f:
#     f.write(cap.stdout)

making init decomposition result: 113.14442777633667

 >> onlinecp rank-50 n_iter-1 mu-1 transformed-True
init_time: 0.8057777881622314
1th_iter: 2.241257905960083 14075.382336308752
2th_iter: 2.381404399871826 17384.540737909476
3th_iter: 4.642629146575928 21765.385084274665
4th_iter: 5.341668367385864 21450.376510630227
5th_iter: 6.681004524230957 18951.459558767412
6th_iter: 9.10312557220459 17905.72513325482
7th_iter: 6.875404119491577 17573.055641402167
8th_iter: 10.52118182182312 17241.379834302392
9th_iter: 10.354374408721924 16465.461775058473
10th_iter: 10.69462537765503 15857.309414858932
11th_iter: 10.427537441253662 18246.465547813437
12th_iter: 10.080900192260742 18445.031382627127
13th_iter: 12.339219331741333 17105.000332946758
14th_iter: 13.206754207611084 15852.116287621775
15th_iter: 13.245729923248291 16192.007508173374
16th_iter: 13.902455806732178 14985.57931766045
17th_iter: 14.675584077835083 14667.19603015106
18th_iter: 15.29148268699646 19120.21639641973
19th_i

In [318]:
# %%capture cap --no-stderr
rank = 50
(weights, factors) = online_tensor_decomposition(X_stream, X, rank, n_iter=1, verbose=False, method='dtd')

X_est = construct_tensor(factors)
compare_tensors(X, X_est)
print_tensor(np.asarray((X, X_est))[:,100,100,100:110,0])
make_video(X_est, './sample_video/dtd-{}.avi'.format(rank))

# with open('./sample_video/dtd.txt', 'w') as f:
#     f.write(cap.stdout)

making init decomposition result: 206.6971755027771

 >> dtd rank-50 n_iter-1 mu-1
1th_iter: 1.208920955657959 14304.171623220953
2th_iter: 2.26269793510437 17611.15088458594
3th_iter: 1.90757417678833 21708.925181998085
4th_iter: 2.3649284839630127 21295.523056015627
5th_iter: 2.9522385597229004 19529.35820676648
6th_iter: 3.0638010501861572 18793.976575859273
7th_iter: 3.8008995056152344 18222.973180943252
8th_iter: 3.223233222961426 17737.563020205052
9th_iter: 3.1701223850250244 17025.272969949317
10th_iter: 2.556931734085083 16565.755112119543
11th_iter: 2.214097023010254 19352.960535295435
12th_iter: 2.5342228412628174 18600.155006344867
13th_iter: 3.7404701709747314 17132.221727778706
14th_iter: 3.4737191200256348 16390.42065758475
15th_iter: 2.3413074016571045 16757.242792809207
16th_iter: 2.942134380340576 15796.957597130478
17th_iter: 2.223156690597534 15435.99258558976
18th_iter: 2.84073543548584 19456.933220855983
19th_iter: 3.2521109580993652 23632.85804524604
20th_iter: 3